In [1]:
import json
import requests
import re
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup, BeautifulStoneSoup

import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()


client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    

In [6]:
df_already = pd.read_sql("SELECT OTHERS FROM TABLE_BUZZ where SOURCE = 'dcinside_미국주식갤러리' ORDER BY DATETIME desc limit 1",conn)


start_id = eval(df_already['OTHERS'][0])['boards_ID']+ 1 #4630000#
start_id

4000722

In [7]:
df_already = pd.read_sql("SELECT OTHERS FROM TABLE_BUZZ where SOURCE = 'dcinside_미국주식갤러리' ORDER BY DATETIME desc limit 1",conn)


start_id = eval(df_already['OTHERS'][0])['boards_ID']+ 1 #4630000#
start_id

err_cnt = 0
boards_ID_l = []
boards_title_l = []
board_main_l = []
board_read_l = []
board_recommend_l = []
board_reply_l = []
board_writer_l = []
board_writerip_l = []
board_time_l = []
board_crawl_time_l = []
board_url_l = []
while True:
    try:

        headers = [
        {'User-Agent' : '' },

        ]
        BASE_URL = 'https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=' + str(start_id)
        response = requests.get(BASE_URL,  headers=headers[0])

        soup = BeautifulSoup(response.content, 'html.parser')
        boards_ID = start_id
        boards_title    = soup.find('title').text.replace(' - 미국 주식 마이너 갤러리','')
        board_main      = soup.find('div', {"class": "write_div"}).text.replace('\n','')
        board_read      = int(soup.find('span', {"class": "gall_count"}).text.replace('조회 ',''))
        board_recommend = int(soup.find('span', {"class": "gall_reply_num"}).text.replace('추천 ',''))
        board_reply     = int(soup.find('span', {"class": "gall_comment"}).text.replace('댓글 ',''))
        board_writer    = soup.find('span', {"class": "nickname"}).text
        board_writerip  = soup.find('span', {"class": "ip"}).text
        board_time      = soup.find('span', {"class": "gall_date"}).text
        board_url= BASE_URL
        print(boards_ID, "//", boards_title, '/',board_main, '/',board_read, '/',board_recommend , '/',board_reply , '/',board_writer, '/',board_url, '/',board_time, '/',
          )
        start_id += 1

        boards_ID_l.append(boards_ID) 
        boards_title_l.append(boards_title) 
        board_main_l.append(board_main) 
        board_read_l.append(board_read) 
        board_recommend_l.append(board_recommend) 
        board_reply_l.append(board_reply) 
        board_writer_l.append(board_writer) 
        board_writerip_l.append(board_writerip) 
        board_time_l.append(board_time) 
        board_crawl_time_l.append(datetime.now())
        board_url_l.append(board_url)


        err_cnt = 0
    except:
        print(start_id,'err')
        err_cnt +=1
        start_id += 1


        if err_cnt > 50:
            break
    time.sleep(0.7)


4000722 err
4000723 // 아 씨발 익절할껄 / 좆같다  / 66 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000723 / 2022.08.31 10:29:33 /
4000724 // 태풍 몇시에 오냐 / 집 창문 열어놨는데  / 50 / 1 / 2 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000724 / 2022.08.31 10:30:19 /
4000725 // 근데 백마들은 전부 핑크임?? / 4!궁금해- dc official App / 116 / 0 / 3 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000725 / 2022.08.31 10:30:36 /
4000726 // 솔직히 지금 미국 좀 빠졌다그래도 하나도 걱정안됨 / 왜냐고?나스닥은 신이니까 / 33 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000726 / 2022.08.31 10:31:20 /
4000727 // 인싸들 이시간에도 섹스하고있겠지? / 진짜 화나네 ㅂㄷㅂㄷ  - dc official App / 159 / 0 / 2 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000727 / 2022.08.31 10:31:31 /
4000728 // 아이돌 산업이 진정한 플랫폼 산업인 것 같음 / 한국이 만든 시스템 안에 일본인이든 한국인이든 중국인이든 집어넣고 똑같이 키워냄외국에서도 따라하기 시작함 저출산 좃도 영향 없을 듯 / 59 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stock

4000768 // 연준이랑 월가랑 짜고치는 고스톱이다 / 위로 좀 오르든 내리든 최종 방향은 이미 정해졌다확신을 가지고 견뎌라 / 61 / 0 / 0 / E피알갱갱 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000768 / 2022.08.31 10:45:20 /
4000769 // 페이팔이란 주식 괜찮아보인다 / 이정도면 몰빵해볼만한데 / 39 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000769 / 2022.08.31 10:45:36 /
4000770 // 한1남들이 얼마나싫엇으면 태풍 변화구틀어서 조지러오노ㅠㅠ / 메갈이보냇나- dc official App / 39 / 0 / 0 / 메타버스인버스 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000770 / 2022.08.31 10:45:37 /
4000771 // 이말올 선넘는데? / 숏충이 다뒤지겟네  - dc official App / 46 / 0 / 0 / 루나노바 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000771 / 2022.08.31 10:46:16 /
4000772 // 월 400충 500충들은 결혼 못한다 / 돈없어서 도태각이다월 600부터 결혼희망 품어볼수있다(세후) / 169 / 0 / 10 / 인텔빠 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000772 / 2022.08.31 10:46:29 /
4000773 // 코인하던 사람들은 그나마 레버타는게 나음 / 변동성은 코인만 못하지만 초보한테 그나마 적은 리스크로  이정도 먹여주는 종목은 없다고봄  - dc official App / 73 / 0 / 0 /

4000810 // 월 300 400벌면 노후에 거지된다니까 / 사람들 다 월 300, 400번다고?그래서 다 거지되잖아대한민국 노인 빈곤율 50프로야평범하게 살면 국평오고나이들어 거지된다고 / 151 / 1 / 4 / 인텔빠 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000810 / 2022.08.31 10:58:00 /
4000811 // 개나소나 하방보는데 떨어지겠냐 / 재무부 할매도 연준 할배들도 하방보는데 떨어지겠냐고 ㅋㅋ / 79 / 0 / 3 / ULTRAJETS / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000811 / 2022.08.31 10:58:03 /
4000812 //  누가 칼들고 공무원 하라고 협박함? / 누가 칼들고 공무원 하지말라고 협박함?  좋으면 하고 아님 하지말고 인터넷백수들이 남 직업에 시애.미질 존나 하네   / 52 / 0 / 1 / <b>ㅇㅇ</b&am / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000812 / 2022.08.31 10:58:03 /
4000813 // 근데 공무원 좆될거는 뻔히 보였는데 왜 그렇게 지원자가 많았을까 / 차라리 일찌감치 코인충이었으면 몇억 벌었을텐데  / 96 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000813 / 2022.08.31 10:58:36 /
4000814 // 삼전 개인 존나사들이고있네ㅋㅋㅋ / 기관 외국인들 물량 다흡수하노- dc official App / 101 / 0 / 0 / 메타버스인버스 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000814 / 2022.08.31 10:58:46 /
4

4000854 // 속슬 10달러 다시오면 산다 안산다 / ㅇㅇ? / 105 / 0 / 3 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000854 / 2022.08.31 11:14:34 /
4000855 // 내년즈음 사기업 웰빙충들 입꾹닫 할거 같노 / 월급도 많이 받는데.정년도 보장된다고 꺼드런대던삼전 /하닉충들 입꾹닫 드갈거 같노 ㅋㅋㅋ10년전만해도 플랜트 / 중공업이 젤 잘나갔는데5년만에 망가진건 기억 못하나 보노 ㅋㅋ / 41 / 0 / 0 / ㅂㅈㄱ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000855 / 2022.08.31 11:14:36 /
4000856 // 나 숏충이... / 어젠 보합이나 약반등 올줄 알고 그냥 나스닥 안봤는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이게뭐고 또 먹었넼ㅋㅋㅋㄴㅋ / 33 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000856 / 2022.08.31 11:15:19 /
4000857 // 오늘 짜장면일 확률 70% / 내 생각임 / 47 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000857 / 2022.08.31 11:15:40 /
4000858 // 삼전지금 블라인드에서 대규모정리 소문난거 아님? / ㅇㅇ?  / 185 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000858 / 2022.08.31 11:15:41 /
4000859 // 우리집 생활비 600정도 들어감 / 세금 연금 보험 기타 모든 비용 포함해서 600아이없음. 강아지 1 있음혼자벌든 둘이벌든 최소 세후 1000 수입이 돼야 강아지라도

4000896 // 개별주 팩트) -80맞아도 잘골랐으면 투베거도 감 / 너가 피터린치면 익절가능ㄷㄷ 파이팅 / 45 / 0 / 0 / Towa / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000896 / 2022.08.31 11:32:15 /
4000897 // 속스 46.9불인데 좆된거냐 / 나 무섭다 탈출시켜줘- dc official App / 95 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000897 / 2022.08.31 11:32:32 /
4000898 // 객관적으로 봐도 나라 망한거 같다 / 세대 갈등 성 갈등 지역 갈등 그런거 의미 없음 다같이 좆됨 / 74 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000898 / 2022.08.31 11:32:39 /
4000899 // 론스타가 멀 잘못했다고 / 론스타 IMF때 위기에 빠진 은행을 인수해 정상화되닌까 판걸가지고 어처구니 업는 한국인들지들이 날려먹은 대우 같은 수없는 기업들은 아무 문제 없당깨 / 103 / 2 / 3 / 가을보리차 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000899 / 2022.08.31 11:33:07 /
4000900 // 솔직히 지금부턴 롱 타도 됨 / 피보나치로 볼때 벌써 50퍼센트 하락 나왔고앞으로33.6퍼센트 남았는데사실 지금부터는 롱타도 되는 자리임 / 76 / 0 / 0 / 트롬프 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4000900 / 2022.08.31 11:33:13 /
4000901 // JEPI 8/29 포폴 TOP10 / 소소하게 바뀜. 브리스톨 

...

4001304 // 난 스큐 80 갈때까지 존버한다. 그날은온다 대곰탕 / . / 27 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001304 / 2022.08.31 13:44:48 /
4001305 err
4001306 // 출근해서 나선 힐끔 보는데 월가는 악마다 그냥 / 9K에서 풀매수 할려고 ww / 203 / 1 / 2 / E4KPT / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001306 / 2022.08.31 13:45:12 /
4001307 err
4001308 // 황스피 +0.5% ㄹㅇ 개쩌네 ㅋㅋㅋㅋㅋㅋㅋ / 어제 1% 오늘 0.5% ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ씹스닥 산 롱숭이들만 벌받고있노 ㅋㅋㅋㅋ / 90 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001308 / 2022.08.31 13:45:58 /
4001309 // 아니 국장은 먼데오름?? / 존나신기하네  / 41 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001309 / 2022.08.31 13:46:03 /
4001310 // 나 숏충이 인데 어제는 롱일줄알았음 / 프장시작했을때 못팔고 좆된줄알았지근데 어제 올랐음오늘도 당연히 롱이겠지 생각하는데장초반에 팔수있음팔고 쏠쓸로 스위칭하려고함근데 주식이 맨날 내생각 반대로라서 두려움- dc official App / 40 / 0 / 0 / 앗싼가오리 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001310 / 2022.08.31 13:46:17 /
4001311 // 코스피오르는이유가머냐대체 / 이해가안되노..  / 10

4001347 // 환율떨어져?  달러더사라무조건1400간다 / 갈  - dc official App / 45 / 1 / 0 / 만출백환 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001347 / 2022.08.31 13:56:24 /
4001348 // 아 좆올맨이 롱처서 불안한데 ㅋㅋㅋㅋ / ㅋㅋㅋㅋ  - dc official App / 32 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001348 / 2022.08.31 13:57:36 /
4001349 // 응 오늘 롱이야 숏충이 다 뒤져 컄ㅋㅋㅋㅋㅋㅋ / 뱉어내라고 컄ㅋㅋㅋ  / 20 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001349 / 2022.08.31 13:58:19 /
4001350 // 미주갤꼬라지보니 달러팔아야겠다 ㅋㅋㅋㅋ / 전럄매도 ㅋㅋㅋ  - dc official App / 37 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001350 / 2022.08.31 13:58:49 /
4001351 // 사실 우린 익어가는 개구리아니냐 / 느낌 쌔한디  / 104 / 2 / 3 / 물음표 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001351 / 2022.08.31 13:59:14 /
4001352 // 폮동각 슬슬 나오노  /  - dc official App / 77 / 1 / 0 / 가치투자자충 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001352 / 2022.08.31 13:59:15 /
40

4001392 // 호로구라양봉에 속지 마라 / 어제도 그랬음  / 29 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001392 / 2022.08.31 14:10:56 /
4001393 // 주식팔면 달러로들어옴? 원화로들어옴? / ??- dc official App / 66 / 0 / 4 / 양마메 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001393 / 2022.08.31 14:10:57 /
4001394 // 현재 시장이 보는 뷰 / 인플레 일시적이다 응 아니야 파멸적 금리인상까꾸라지는 시장 / 37 / 0 / 0 / ㅇㅅㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001394 / 2022.08.31 14:11:17 /
4001395 // 여자 챙녀인거 알고도 만나는거임 / 그냥 만나면 생글생글 웃고이것저곳 돌아다니면서 엥기고 섹스하면 좋은걸 어떡하냐 ㅠㅠ남자로 태어난 업보임 / 71 / 0 / 4 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001395 / 2022.08.31 14:11:30 /
4001396 // 킹스피 프로그램 매수 미쳤누 / 비차익 800억은 본적도 없는 숫자인데...ㅋㅋㅋㅋ 뭐냐 이거장마감에 다 던질려나 그럴 액수가 아닌거 같은데 / 176 / 1 / 4 / DirexionDaily3XEtf / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001396 / 2022.08.31 14:11:27 /
4001397 // 난 세상 모든 피싸개들의 천적임... / 피싸개 씹극혐 / 26 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/m

4001439 // 한동1훈 <~ 론스타 이긴거 개지리는거임? / - dc official App / 101 / 1 / 7 / 호랑이 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001439 / 2022.08.31 14:24:51 /
4001440 // 시발...라부탈껄 / 안전빵 노리다가 종목선택 미스~ / 70 / 0 / 1 / DirexionDaily3XEtf / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001440 / 2022.08.31 14:25:33 /
4001441 // 아버지 직업으로 개업 세무사 vs. 행시 2~3급 공무원 / 본인 아버지 직업으로강남에 개업해서 세무법인 또는 사무실 운영하는 대표 세무사연봉 순이익 2억vs.행시 출신으로 세종 기재부에서 근무하시다 2~3급 명예퇴직그리고 현재 연금 수급중둘 중 어느 분이 본인 아버지였으면 좋겠음?세무사와 행시출신의 평균적인 60대 모습인데 / 34 / 0 / 1 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001441 / 2022.08.31 14:25:34 /
4001442 // 환율 말좆음봉 뭐노ㅋㅋㅋ / 버스 지금타도 되냐  / 35 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001442 / 2022.08.31 14:25:38 /
4001443 // 또또또 연준에 대항하지 / 싸가지가 없는거지 이건 / 18 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001443 / 2022.08.31 14:25:49 /
4001444 // 욕심 그득그득한 숏충이들 수익 다날라가넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

4001487 // 절대 레버리지 하면 안되는 이유 / 올라도 오르는게아님  - dc official App / 69 / 1 / 2 / 가치투자자충 / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001487 / 2022.08.31 14:35:23 /
4001488 // 나선 의미 없다 / 나선에서 오르고 떨어지는 경우가 더 많음 / 23 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001488 / 2022.08.31 14:35:36 /
4001489 // 폭등하쥬? 숏충이들 뜨근하쥬? / 컄ㅋㅋ  / 9 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001489 / 2022.08.31 14:35:37 /
4001490 // 스윙 치기엔 항셍이 더 좋지 않음? / 박스권에서 와리가리치는데 / 10 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001490 / 2022.08.31 14:36:10 /
4001491 // 와 ㄷㄷ 황플 2% 떡상중이다 ㄷㄷㄷㄷ / ㅗ / 38 / 0 / 0 / ㅇㅇ / https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=4001491 / 2022.08.31 14:36:16 /
4001492 // 작년에 넣어서 원금 -35퍼 손실본 애들 있냐? / 유동원 말믿고 존버타다가물린거 그냥 안팔고 있었는데어제 그냥 다팔았음내가 무지하고 멍청한 것에 대해 인정하고스큐사서 어제 4퍼 먹고 손절했다오늘 프리장에 스큐 떨어지면 사서 연설 끝나고 1시간 뒤에 팔면서 복구할거임 / 18 / 0 / 2 / ㅇㅇ / https://gall.dcinside.com/

In [8]:
df_ = pd.DataFrame()
df_['boards_ID'] = boards_ID_l 
df_['boards_title'] = boards_title_l 
df_['board_main'] = board_main_l 
df_['board_read'] = board_read_l 
df_['board_recommend'] = board_recommend_l 
df_['board_reply'] = board_reply_l 
df_['board_writer'] = board_writer_l 
df_['board_writerip'] = board_writerip_l 
df_['board_time'] = board_time_l 
df_['board_time'] = df_['board_time'].apply(lambda x :  datetime.strptime(x, '%Y.%m.%d %H:%M:%S')) 
df_['board_crawl_time'] =  board_crawl_time_l
df_['board_url'] =  board_url_l
df_['source'] = 'dc_gall'
df_['updated'] = (datetime.now() + timedelta(hours=9)).strftime("%Y%m%d-%H%M%S")

df_['DATETIME'] = df_['board_time'].apply(lambda x :  x.strftime("%Y%m%d-%H%M%S") ) 
df_['VIEWS'] = df_['board_read']
df_['LIKES'] = df_['board_recommend']
df_['TITLE'] =  df_['boards_title']
df_['CONTENT'] = df_['board_main']
df_['WRITER'] =df_['board_writer']
df_['SOURCE'] = "dcinside_미국주식갤러리"
df_['RELATED_ITEM'] = np.nan
df_['POSITIVITY'] = np.nan
for i in range(len(df_)):
    df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    
df_['OTHERS'] = df_[['boards_ID','board_reply','board_writerip']].to_dict('records')
df_['OTHERS'] = df_['OTHERS'].astype(str)

df_['NUM_REPLY'] = df_['board_reply'] 
df_['CRAWLING_DATETIME'] =df_['updated'] 
df_['URL'] =  df_['board_url']
df_ttl = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]

df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,dcinside_미국주식갤러리,NaN,20220831-102933,아 씨발 익절할껄,좆같다,0.003790,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4000723, 'board_reply': 0, 'boar..."
1,dcinside_미국주식갤러리,NaN,20220831-103019,태풍 몇시에 오냐,집 창문 열어놨는데,0.065275,2,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4000724, 'board_reply': 2, 'boar..."
2,dcinside_미국주식갤러리,NaN,20220831-103036,근데 백마들은 전부 핑크임??,4!궁금해- dc official App,0.065724,3,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4000725, 'board_reply': 3, 'boar..."
3,dcinside_미국주식갤러리,NaN,20220831-103120,솔직히 지금 미국 좀 빠졌다그래도 하나도 걱정안됨,왜냐고?나스닥은 신이니까,8.929663,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4000726, 'board_reply': 0, 'boar..."
4,dcinside_미국주식갤러리,NaN,20220831-103131,인싸들 이시간에도 섹스하고있겠지?,진짜 화나네 ㅂㄷㅂㄷ - dc official App,0.005901,2,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4000727, 'board_reply': 2, 'boar..."
...,...,...,...,...,...,...,...,...,...,...
741,dcinside_미국주식갤러리,NaN,20220831-143830,아시아가 주도하는 세곜ㅋㅋㅋㅋㅋㅋㅅ,숏충이 뒤졐ㅋㅋㅋㅋ,0.192963,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4001502, 'board_reply': 0, 'boar..."
742,dcinside_미국주식갤러리,NaN,20220831-143833,"자 숏충이님들 따라해봅시다 ""선 반 영""",선반영 ^,0.054832,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4001503, 'board_reply': 0, 'boar..."
743,dcinside_미국주식갤러리,NaN,20220831-143853,근데 한국국적 포기하고싶으면 비정상임?,"뭐 존나 옛날부터 들었던 생각이긴 하지만,자칭 선진국이랍시고 거들먹거리면서도 막상 ...",0.004179,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4001504, 'board_reply': 0, 'boar..."
744,dcinside_미국주식갤러리,NaN,20220831-143859,으앜 나스닥 배부르다 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,돈복사 ㅋㅋㅋㅋ - dc official App,0.691776,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-144000,"{'boards_ID': 4001505, 'board_reply': 0, 'boar..."


In [4]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " DC_US-STOCK , SUCCESS  : ",len(df_ttl))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " DC_US-STOCK , NO DATA TO UPDATE")

20220831-142347  DC_US-STOCK , SUCCESS  :  628


In [9]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " DC_US-STOCK , SUCCESS  : ",len(df_ttl))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " DC_US-STOCK , NO DATA TO UPDATE")

20220831-144429  DC_US-STOCK , SUCCESS  :  746


In [68]:
df_ttl = pd.DataFrame()

In [69]:
df_ = pd.DataFrame()
df_['boards_ID'] = boards_ID_l 
df_['boards_title'] = boards_title_l 
df_['board_main'] = board_main_l 
df_['board_read'] = board_read_l 
df_['board_recommend'] = board_recommend_l 
df_['board_reply'] = board_reply_l 
df_['board_writer'] = board_writer_l 
df_['board_writerip'] = board_writerip_l 
df_['board_time'] = board_time_l 
df_['board_time'] = df_['board_time'].apply(lambda x :  datetime.strptime(x, '%Y.%m.%d %H:%M:%S')) 
df_['board_crawl_time'] =  board_crawl_time_l
df_['board_url'] =  board_url_l
df_['source'] = 'dc_gall'
df_['updated'] = (datetime.now() + timedelta(hours=9)).strftime("%Y%m%d-%H%M%S")

df_['DATETIME'] = df_['board_time'].apply(lambda x :  x.strftime("%Y%m%d-%H%M%S") ) 
df_['VIEWS'] = df_['board_read']
df_['LIKES'] = df_['board_recommend']
df_['TITLE'] =  df_['boards_title']
df_['CONTENT'] = df_['board_main']
df_['WRITER'] =df_['board_writer']
df_['SOURCE'] = "dcinside_해외주식갤러리"
df_['RELATED_ITEM'] = np.nan
df_['POSITIVITY'] = np.nan
for i in range(len(df_)):
    df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    
df_['OTHERS'] = df_[['boards_ID','board_reply','board_writerip']].to_dict('records')
df_['OTHERS'] = df_['OTHERS'].astype(str)

df_['NUM_REPLY'] = df_['board_reply'] 
df_['CRAWLING_DATETIME'] =df_['updated'] 
df_['URL'] =  df_['board_url']
df_ttl = df_ttl.append(df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]])
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,dcinside_해외주식갤러리,NaN,20220830-153043,BBBY offering 31 Aug confirmed - 해외주식 마이너 갤러리,ㅅㄱ - dc official App,0.025226,9,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4636841, 'board_reply': 9, 'boar..."
1,dcinside_해외주식갤러리,NaN,20220830-153136,avct 121.184 억까새끼 어디갔노 - 해외주식 마이너 갤러리,오르니깐 싸물고 있네 ㅋㅋ - dc official App,0.007236,2,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4636842, 'board_reply': 2, 'boar..."
2,dcinside_해외주식갤러리,NaN,20220830-153353,bbby 오퍼링이네... - 해외주식 마이너 갤러리,하.. - dc official App,0.094986,4,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4636843, 'board_reply': 4, 'boar..."
3,dcinside_해외주식갤러리,NaN,20220830-153448,비비비와이 유증 확정 - 해외주식 마이너 갤러리,하 - dc official App,0.034671,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4636844, 'board_reply': 0, 'boar..."
4,dcinside_해외주식갤러리,NaN,20220830-153453,bbby 오퍼링ㅋㅋㅋㅋㅋ - 해외주식 마이너 갤러리,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0.071702,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4636845, 'board_reply': 0, 'boar..."
...,...,...,...,...,...,...,...,...,...,...
2013,dcinside_해외주식갤러리,NaN,20220831-104647,BBBY 오늘 발표 몇시임? - 해외주식 마이너 갤러리,21:30이었나 - dc official App,0.025760,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4638980, 'board_reply': 0, 'boar..."
2014,dcinside_해외주식갤러리,NaN,20220831-104849,오늘 스타박스 반등 구간이다 무조건 타라 - 해외주식 마이너 갤러리,안타면 너 손해- dc official App,0.854511,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4638981, 'board_reply': 1, 'boar..."
2015,dcinside_해외주식갤러리,NaN,20220831-105427,와 nogn - 해외주식 마이너 갤러리,Onfo 뮬타다가 손절하고 그거 그대로nogn 4.45에 넣고 새벽 4시에 onfo...,0.039104,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4638982, 'board_reply': 1, 'boar..."
2016,dcinside_해외주식갤러리,NaN,20220831-105505,나 윤모 찐 맞음 본캐 아이피 차단당해서 - 해외주식 마이너 갤러리,통피로 글올리는중,0.006474,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-105936,"{'boards_ID': 4638983, 'board_reply': 0, 'boar..."


In [75]:
eval("{'boards_ID': '4630000' , 'sd':'sdd'}")['']

{'boards_ID': '4630000', 'sd': 'sdd'}

In [78]:
df_already = pd.read_sql("SELECT OTHERS FROM TABLE_BUZZ where SOURCE = 'dcinside_해외주식갤러리' ORDER BY DATETIME desc limit 1",conn)
eval(df_already['OTHERS'][0])['boards_ID']


4638984

In [ ]:
while True:
    max_id = pd.read_sql('SELECT max(boards_ID) FROM STOCK_GALLERY',conn)
    
    start_id = max_id.iloc[0,0] +1
    err_cnt = 0
    boards_ID_l = []
    boards_title_l = []
    board_main_l = []
    board_read_l = []
    board_recommend_l = []
    board_reply_l = []
    board_writer_l = []
    board_writerip_l = []
    board_time_l = []
    board_crawl_time_l = []
    board_url_l = []
    while True:
        try:

            headers = [
            {'User-Agent' : '' },

            ]
            BASE_URL = 'https://gall.dcinside.com/board/view/?id=neostock&no=' + str(start_id)
            response = requests.get(BASE_URL,  headers=headers[0])

            soup = BeautifulSoup(response.content, 'html.parser')
            boards_ID = start_id
            boards_title    = soup.find('title').text.replace(' - 비트코인 갤러리','')
            board_main      = soup.find('div', {"class": "write_div"}).text.replace('\n','')
            board_read      = int(soup.find('span', {"class": "gall_count"}).text.replace('조회 ',''))
            board_recommend = int(soup.find('span', {"class": "gall_reply_num"}).text.replace('추천 ',''))
            board_reply     = int(soup.find('span', {"class": "gall_comment"}).text.replace('댓글 ',''))
            board_writer    = soup.find('span', {"class": "nickname"}).text
            board_writerip  = soup.find('span', {"class": "ip"}).text
            board_time      = soup.find('span', {"class": "gall_date"}).text
            board_url= BASE_URL
            print(boards_ID, "//", boards_title, '/',board_main, '/',board_read, '/',board_recommend , '/',board_reply , '/',board_writer, '/',board_url, '/',board_time, '/',
              )
            start_id += 1

            boards_ID_l.append(boards_ID) 
            boards_title_l.append(boards_title) 
            board_main_l.append(board_main) 
            board_read_l.append(board_read) 
            board_recommend_l.append(board_recommend) 
            board_reply_l.append(board_reply) 
            board_writer_l.append(board_writer) 
            board_writerip_l.append(board_writerip) 
            board_time_l.append(board_time) 
            board_crawl_time_l.append(datetime.datetime.now())
            board_url_l.append(board_url)


            err_cnt = 0
        except:
            print(start_id,'err')
            err_cnt +=1
            start_id += 1


            if err_cnt > 100:
                break
        time.sleep(0.7)
        
    df_board = pd.DataFrame()
    df_board['boards_ID'] = boards_ID_l 
    df_board['boards_title'] = boards_title_l 
    df_board['board_main'] = board_main_l 
    df_board['board_read'] = board_read_l 
    df_board['board_recommend'] = board_recommend_l 
    df_board['board_reply'] = board_reply_l 
    df_board['board_writer'] = board_writer_l 
    df_board['board_writerip'] = board_writerip_l 
    df_board['board_time'] = board_time_l 
    df_board['board_time'] = df_board['board_time'].apply(lambda x :  datetime.datetime.strptime(x, '%Y.%m.%d %H:%M:%S')) 
    df_board['board_crawl_time'] =  board_crawl_time_l
    df_board['board_url'] =  board_url_l
    df_board['source'] = 'dc_gall'

    df_board['updated'] = (datetime.datetime.now() + datetime.timedelta(hours=9)).strftime("%Y%m%d-%H%M%S")
            
    df_board.to_sql('STOCK_GALLERY',conn,if_exists = 'append',index = False)
    print(datetime.datetime.now(), " CRAWL COMPLETED================== inserted data : ",len(df_board) )
    time.sleep(60 * 60)

In [66]:
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,dcinside_해외주식갤러리,NaN,20220825-124205,(홀)(짝)(홀)(짝)(홀)(?) - 해외주식 마이너 갤러리,아직 안튄 흑우없제?,0.017420,3,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4630000, 'board_reply': 3, 'boar..."
1,dcinside_해외주식갤러리,NaN,20220825-125409,월급받고 매일 5만원씩 달러환전 어케생각하냐 - 해외주식 마이너 갤러리,27 계약 3200 - dc official App,0.007051,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4630001, 'board_reply': 1, 'boar..."
2,dcinside_해외주식갤러리,NaN,20220825-125616,"AMC,APE 오늘 장 예상 - 해외주식 마이너 갤러리",오늘자로 한국 주주들도 APE 배당받았고 다른나라도 T3적용받으면 대부분이 오늘 받...,0.020652,14,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4630002, 'board_reply': 14, 'boa..."
3,dcinside_해외주식갤러리,NaN,20220825-125712,야스각은 살아있다 - 해외주식 마이너 갤러리,,0.101114,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4630003, 'board_reply': 1, 'boar..."
4,dcinside_해외주식갤러리,NaN,20220825-125828,오늘 관심가질만한건 없나? - 해외주식 마이너 갤러리,요근래 오늘은 언급되는게 별로 없네ㅋㅋㅋ 새로운 티커의 시작인가,0.011893,6,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4630004, 'board_reply': 6, 'boar..."
...,...,...,...,...,...,...,...,...,...,...
8262,dcinside_해외주식갤러리,NaN,20220831-092648,환율때메 멀사길 망설여 지네 - 해외주식 마이너 갤러리,ㅅㅂ,0.006399,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4638960, 'board_reply': 1, 'boar..."
8263,dcinside_해외주식갤러리,NaN,20220831-092904,Ahi 는 뭐 얼마나 대단한짓하길래 거래정지냐 - 해외주식 마이너 갤러리,별것도 없을거 같은데 존나 떨구기만해봐 ㅅㅂ,0.007065,0,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4638961, 'board_reply': 0, 'boar..."
8264,dcinside_해외주식갤러리,NaN,20220831-093542,BBBY 병.신.들.아. 내 말들어봐ㅋㅋ - 해외주식 마이너 갤러리,소문난 잔칫상에 먹을 거 없다.뉴스에 팔아라. 적어도 발표 2일전은 털으라했다.다신...,0.010902,1,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4638962, 'board_reply': 1, 'boar..."
8265,dcinside_해외주식갤러리,NaN,20220831-093736,해주 친구들은 속고만 살았니 - 해외주식 마이너 갤러리,형아가 앞으로 급등주 미리미리 파악해서 알려줄게단!톡에 있으니 답답해서 하산 한건데...,0.007481,8,https://gall.dcinside.com/mgallery/board/view/...,20220831-094440,"{'boards_ID': 4638963, 'board_reply': 8, 'boar..."


In [71]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " NO DATA TO UPDATE")

20220831-135058  SUCCESS  :  2018
